### Extended Figure 2 - Effect of Long Capsule Induction on Growth Rate. 

In this Notebook we plot and analyze the effect of a long >10 doublings induction of MTCs on doubling rate.  This dataset includes an earlier version of the MTC strain (MP010) that had a much higher rate of induction.  The strain which is plotted (denoted here as "flip") corresponds to MP050 and is the strain used throughout this publication. 

Note - this notebook makes use of bokeh's export svg functionality to create svgs of each image to inlcude in Adobe illustrator.  However each figure is also generated as a preview in Jupyter notebook.  Simply don't run cells that save the image to svg if this is an issue for you and you should still be able to preview the interactive figures. 

In [1]:
import pandas as pd
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg, curdoc
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, ColorBar
import datetime
from scipy.stats import gaussian_kde
import numpy as np
from sklearn.metrics import r2_score
from bokeh.themes import Theme
curdoc().theme = Theme(filename="../../figure_theme.yaml")

output_notebook()

Loading BokehJS ...

In [2]:
data_location = "..\\..\\Non Sequencing Raw Data and Experiment Notes\\2.13.23_growth_data.csv"
data_df = pd.read_csv(data_location, sep = '\t')
for key in data_df.columns:
    data_df[key] = data_df[key].apply(lambda x: np.nan if x=="____" else float(x))

In [15]:
p = figure(width=775, height=775, y_axis_type="log",
          output_backend = "svg", x_axis_label = 'Time (Min)',
           y_axis_label = 'Optical Density (OD 600)',)
colors = ["olivedrab", "mediumturquoise", "maroon", "palevioletred"]

y_vals = ["WT_I", "WT_U", "flip_I", "flip_U"]
thresh = 0.018 #threshold on OD to use as the zero time

doubling_time = []
for val in y_vals:
    first_od_over_thresh = min(data_df[data_df[val] > thresh][val])
    time_adj = float(data_df[data_df[val] == first_od_over_thresh]["Time(minutes)"])
    time = [x - time_adj for i,x in enumerate(data_df["Time(minutes)"]) if ~np.isnan(data_df[val][i]) and x - time_adj >= 0]
    values = [np.log2(x) for i, x in enumerate(data_df[val]) if ~np.isnan(x) and data_df["Time(minutes)"][i] - time_adj >= 0]
    m, _ = np.polyfit(time, values, 1)
    doubling_time.append(1./m)
    


names = [
    f"WT +IPTG, t = {doubling_time[0]:.1f} min",
    f"WT -IPTG, t = {doubling_time[1]:.1f} min",
    f"MTC +IPTG, t = {doubling_time[2]:.1f} min",
    f"MTC -IPTG, t = {doubling_time[3]:.1f} min",
]

for ind, val in enumerate(y_vals):
    first_od_over_thresh = min(data_df[data_df[val] > thresh][val])
    time_adj = float(data_df[data_df[val] == first_od_over_thresh]["Time(minutes)"])
    time = [x - time_adj for i,x in enumerate(data_df["Time(minutes)"]) if ~np.isnan(data_df[val][i]) and x - time_adj >= 0]
    values = [x for i, x in enumerate(data_df[val]) if ~np.isnan(x) and data_df["Time(minutes)"][i] - time_adj >= 0]
    p.line(time, values, color = colors[ind], line_width = 5)
    p.circle(time, values, legend_label = names[ind], color = colors[ind], size = 10)

p.legend.location = "top_left"
p.legend.label_text_font_size = '28pt'
p.legend.label_text_font_style = "normal"
p.legend.label_text_color = '#000000'
p.background_fill_color = "white"
p.grid.grid_line_color = None

p.axis.major_label_text_color = '#000000'
p.axis.major_label_text_font_size = "32px"
p.axis.major_label_text_font_style = "normal"
p.axis.major_label_text_font = "arial"

p.axis.axis_label_text_color = '#000000'
p.axis.axis_label_text_font_style = "normal"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_size = "36px"
p.yaxis.ticker = [0.025, 0.05, 0.1, 0.20]
p.xaxis.ticker = [0, 20, 40, 60, 80]
show(p)

In [16]:
export_svg(p, filename = f'./Extended_Figure_2_{datetime.date.today()}.svg')

['./Extended_Figure_2_2023-10-09.svg']